**Exploratory data analysis to understand the distribution of sensor readings over time and their relationship with room occupancy**

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('/data/notebook_files/Occupancy_Estimation.csv')
df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

df.set_index('Timestamp', inplace=True)

plt.figure(figsize=(12, 6))
sns.lineplot(x='Timestamp', y='S1_Temp', data=df, label='S1_Temp')
sns.lineplot(x='Timestamp', y='Room_Occupancy_Count', data=df, label='Room Occupancy')
plt.plot(df['S1_Temp'], label='Sensor Reading', color='blue')
plt.title('Sensor Readings Over Time')
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.legend()
plt.show()

**Handling missing values and adding a meaningful train/test split**

In [3]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

occupancy_df = pd.read_csv('/data/notebook_files/Occupancy_Estimation.csv')

print("\nTotal Missing Data Before Imputation:")
print(occupancy_df.isnull().sum())

numeric_columns = occupancy_df.select_dtypes(include=['number']).columns
non_numeric_columns = occupancy_df.select_dtypes(exclude=['number']).columns
numeric_imputer = SimpleImputer(strategy='mean')
occupancy_df[numeric_columns] = numeric_imputer.fit_transform(occupancy_df[numeric_columns])
non_numeric_imputer = SimpleImputer(strategy='most_frequent')
occupancy_df[non_numeric_columns] = non_numeric_imputer.fit_transform(occupancy_df[non_numeric_columns])
missing_values_after_imputation = occupancy_df.isnull().sum()

print("\nTotal Missing Data After Imputation:")
print(missing_values_after_imputation)

X = occupancy_df.drop('Room_Occupancy_Count', axis=1)
y = occupancy_df['Room_Occupancy_Count']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



Total Missing Data Before Imputation:
Date                    0
Time                    0
S1_Temp                 0
S2_Temp                 0
S3_Temp                 0
S4_Temp                 0
S1_Light                0
S2_Light                0
S3_Light                0
S4_Light                0
S1_Sound                0
S2_Sound                0
S3_Sound                0
S4_Sound                0
S5_CO2                  0
S5_CO2_Slope            0
S6_PIR                  0
S7_PIR                  0
Room_Occupancy_Count    0
dtype: int64

Total Missing Data After Imputation:
Date                    0
Time                    0
S1_Temp                 0
S2_Temp                 0
S3_Temp                 0
S4_Temp                 0
S1_Light                0
S2_Light                0
S3_Light                0
S4_Light                0
S1_Sound                0
S2_Sound                0
S3_Sound                0
S4_Sound                0
S5_CO2                  0
S5_CO2_Slope            0


**Neural network design utilising the temporal nature of the data.**

In [4]:
import torch
import torch.nn as nn

features = ['S1_Temp', 'S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light', 'S2_Light', 'S3_Light', 'S4_Light',
            'S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound', 'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR']

class OccupancyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(OccupancyLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out_last = lstm_out[:, -1, :]
        output = self.fc(lstm_out_last)
        return output

input_size = len(features)
hidden_size = 64 
num_layers = 1
output_size = 1

model = OccupancyLSTM(input_size, hidden_size, num_layers, output_size)

print(model)

OccupancyLSTM(
  (lstm): LSTM(16, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


**Training the neural network using training set**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

features = ['S1_Temp','S2_Temp', 'S3_Temp', 'S4_Temp', 'S1_Light', 'S2_Light', 'S3_Light', 'S4_Light', 'S1_Sound', 'S2_Sound', 'S3_Sound', 'S4_Sound', 'S5_CO2', 'S5_CO2_Slope', 'S6_PIR', 'S7_PIR']

scaler = StandardScaler()
occupancy_df[features] = scaler.fit_transform(occupancy_df[features])

X_tensor = torch.tensor(occupancy_df[features].values, dtype=torch.float32)
y_tensor = torch.tensor(occupancy_df['Room_Occupancy_Count'].values, dtype=torch.float32).view(-1, 1)

class EstimationModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EstimationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = len(features)
hidden_size = 64
model = EstimationModel(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

num_epochs = 50
for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    test_outputs = model(X_tensor)
    test_loss = criterion(test_outputs, y_tensor)

print(f'Test Loss: {test_loss.item():.4f}')

Epoch [1/50], Loss: 0.0216
Epoch [2/50], Loss: 0.0022
Epoch [3/50], Loss: 0.0664
Epoch [4/50], Loss: 0.0016
Epoch [5/50], Loss: 0.0059
Epoch [6/50], Loss: 0.0132
Epoch [7/50], Loss: 0.0039
Epoch [8/50], Loss: 0.0022
Epoch [9/50], Loss: 0.0041
Epoch [10/50], Loss: 0.0050
Epoch [11/50], Loss: 0.0030
Epoch [12/50], Loss: 0.0356
Epoch [13/50], Loss: 0.0049
Epoch [14/50], Loss: 0.0013
Epoch [15/50], Loss: 0.0029
Epoch [16/50], Loss: 0.0013
Epoch [17/50], Loss: 0.0074
Epoch [18/50], Loss: 0.0009
Epoch [19/50], Loss: 0.0462
Epoch [20/50], Loss: 0.0007
Epoch [21/50], Loss: 0.0103
Epoch [22/50], Loss: 0.0019
Epoch [23/50], Loss: 0.0028
Epoch [24/50], Loss: 0.0039
Epoch [25/50], Loss: 0.0020
Epoch [26/50], Loss: 0.0202
Epoch [27/50], Loss: 0.0280
Epoch [28/50], Loss: 0.0011
Epoch [29/50], Loss: 0.0017
Epoch [30/50], Loss: 0.0005
Epoch [31/50], Loss: 0.0144
Epoch [32/50], Loss: 0.0102
Epoch [33/50], Loss: 0.0173
Epoch [34/50], Loss: 0.0005
Epoch [35/50], Loss: 0.0291
Epoch [36/50], Loss: 0.0003
E

**Evaluating model's performance and reporting the training curves and metrics**

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

train_losses = []
test_losses = []
mse_scores = []
r2_scores = []

for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    train_losses.append(loss.item())

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_tensor)
        test_loss = criterion(test_outputs, y_tensor)

    test_losses.append(test_loss.item())

    test_predictions = test_outputs.numpy()
    true_values = y_tensor.numpy()

    mse = mean_squared_error(true_values, test_predictions)
    r2 = r2_score(true_values, test_predictions)

    mse_scores.append(mse)
    r2_scores.append(r2)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}, MSE: {mse:.4f}, R2: {r2:.4f}')

plt.figure(figsize=(12, 6))
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), test_losses, label='Test Loss')
plt.title('Training and Testing Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch [1/50], Train Loss: 0.0031, Test Loss: 0.0049, MSE: 0.0049, R2: 0.9938
Epoch [2/50], Train Loss: 0.0006, Test Loss: 0.0044, MSE: 0.0044, R2: 0.9944
Epoch [3/50], Train Loss: 0.0087, Test Loss: 0.0042, MSE: 0.0042, R2: 0.9947
Epoch [4/50], Train Loss: 0.0019, Test Loss: 0.0049, MSE: 0.0049, R2: 0.9939
Epoch [5/50], Train Loss: 0.0014, Test Loss: 0.0040, MSE: 0.0040, R2: 0.9949
Epoch [6/50], Train Loss: 0.0005, Test Loss: 0.0054, MSE: 0.0054, R2: 0.9932
Epoch [7/50], Train Loss: 0.0018, Test Loss: 0.0044, MSE: 0.0044, R2: 0.9944
Epoch [8/50], Train Loss: 0.0012, Test Loss: 0.0048, MSE: 0.0048, R2: 0.9940
Epoch [9/50], Train Loss: 0.0055, Test Loss: 0.0040, MSE: 0.0040, R2: 0.9950
Epoch [10/50], Train Loss: 0.0004, Test Loss: 0.0044, MSE: 0.0044, R2: 0.9944
Epoch [11/50], Train Loss: 0.0018, Test Loss: 0.0043, MSE: 0.0043, R2: 0.9946
Epoch [12/50], Train Loss: 0.0018, Test Loss: 0.0049, MSE: 0.0049, R2: 0.9938
Epoch [13/50], Train Loss: 0.0011, Test Loss: 0.0036, MSE: 0.0036, R2: 0.

**Question 4.**

In [19]:

# Layer 2: Conv-1: 64 kernels of size 7x7, stride of 2 and padding of 0

def calculate_conv_output_size(input_size, kernel_size, stride, padding):
    return ((input_size - kernel_size + 2 * padding) // stride) + 1

input_size = 256
kernel_size = 7
stride = 2
padding = 0
num_kernels=64
kernel_height = 7
kernel_width = 7
num_input_channels = 3

output_size = calculate_conv_output_size(input_size, kernel_size, stride, padding)
num_weights = num_kernels * kernel_height * kernel_width * num_input_channels
num_biases = num_kernels

print(f"Output size: {output_size}x{output_size}x{num_kernels}")

print(f"Number of weights: {num_weights}")
print(f"Number of biases: {num_biases}")

Output size: 125x125x64
Number of weights: 9408
Number of biases: 64


**Question 7**

In [16]:
import numpy as np
import matplotlib.pyplot as plt

def loss_function(epsilon):
    return np.sum(np.square(2 * np.arctan(np.square(epsilon) + 0.5463) - 1))/5

epsilon_values = np.linspace(0.01, 2.0, 100)

loss_values = np.array([loss_function(epsilon) for epsilon in epsilon_values])

plt.plot(epsilon_values, loss_values, label='Loss Function')
plt.title('Loss Function Visualization')
plt.xlabel('Epsilon (ϵ)')
plt.ylabel('Loss')
plt.legend()
plt.show()